In [2]:
import json
import os
from glob import glob
import pandas as pd
import re
import scipy.stats as st
import numpy as np

In [3]:
def read_experiment_stats(regex):
    all_files = glob("./stats/*.json")
    files = []
    for f in all_files:
        temp = os.path.split(f)[-1]
        if re.search(regex, temp) is not None:
            files.append(f)
        
    method_acc = []
    method_total_flops = []
    method_client_flops = []
    method_comm = []
    steps = []
    names = []

        

    acc_dict = {}
    for i, f in enumerate(files):
        exp_name = os.path.split(f)[-1].replace(".json", "")
        run = exp_name.split("_")[-1][:-1]
        exp_name = "_".join(exp_name.split("_")[:-1])

        j = json.load(open(f, "r"))
        method_acc.append(j['Method Accuracy'])
        method_total_flops.append(j['Method Flops'])
        method_client_flops.append(j['Method Client Flops'])
        method_comm.append(j['Method Comm Cost'])
        steps.append(j['Steps'])
        names.append(os.path.split(f)[-1].replace(".json", ""))
        
        if exp_name not in acc_dict:
            acc_dict[exp_name] = {}
            acc_dict[exp_name]['acc_list'] = []
            acc_dict[exp_name]['flop_list'] = []
            acc_dict[exp_name]['comm_list'] = []
        acc_dict[exp_name]['acc_list'].append(max(eval(j['Method Accuracy'])))
        acc_dict[exp_name]['flop_list'].append(eval(j['Method Client Flops'])[-1] * 1e-12)
        acc_dict[exp_name]['comm_list'].append(eval(j['Method Comm Cost'])[-1] * 1e-3)
        
        # for k, v in j['hparams'].items():
        #     hparams[k].append(v)

    exp_dict = {}
    for k in acc_dict.keys():
        exp_dict[k] = {}
        exp_dict[k]['acc_avg'] = np.mean(acc_dict[k]['acc_list'])
        exp_dict[k]['acc_std'] = np.std(acc_dict[k]['acc_list'])
        exp_dict[k]['cflop_avg'] = np.mean(acc_dict[k]['flop_list'])
        exp_dict[k]['cflop_std'] = np.std(acc_dict[k]['flop_list'])
        exp_dict[k]['comm_avg'] = np.mean(acc_dict[k]['comm_list'])
        exp_dict[k]['comm_std'] = np.std(acc_dict[k]['comm_list'])

    
    df = pd.DataFrame(
        zip(names, method_acc, method_comm, method_total_flops, method_client_flops), 
        columns=["Experiment Name", "Method Accuracy", "Method Comm Cost", "Total Flops", "Client Flops"]
    )
    
    # Make main results table
    table = pd.DataFrame()
    table['Experiment Name'] = df['Experiment Name']
    table['Method Accuracy'] = df['Method Accuracy'].apply(lambda x: max(eval(x)))
    table['Client TFLOPS'] = df['Client Flops'].apply(lambda x: eval(x)[-1] * 1e-12)
    table['Total TFLOPS'] = df['Total Flops'].apply(lambda x: eval(x)[-1] * 1e-12)
    table['Comm Cost (GBs)'] = df['Method Comm Cost'].apply(lambda x: eval(x)[-1] * 1e-3)

    return df, table, exp_dict

def compute_acc_per_comm_per_flop(metric_dict):
    new_dict = {}
    for k in metric_dict.keys():
        new_dict[k] = metric_dict[k]['acc_avg'] / (metric_dict[k]['cflop_avg'] * metric_dict[k]['comm_avg'])
    return new_dict


In [4]:
df, table, metric_dict = read_experiment_stats(r'.*')
metric_dict

{'non_iid_50_CESL-3-5': {'acc_avg': 0.8821470351826651,
  'acc_std': 0.0017214902212495304,
  'cflop_avg': 5.384503296,
  'cflop_std': 0.0,
  'comm_avg': 17.02483476000416,
  'comm_std': 0.01327006493309689},
 'non_iid_50_CESL-No-Head-5-5': {'acc_avg': 0.8421027308299749,
  'acc_std': 0.0032276278672320273,
  'cflop_avg': 3.7601280000000004,
  'cflop_std': 4.440892098500626e-16,
  'comm_avg': 92.14405064482678,
  'comm_std': 0.08619550343644859},
 'non_iid_50_CESL-Random-3-5': {'acc_avg': 0.8806207072717308,
  'acc_std': 0.0016681649397915683,
  'cflop_avg': 5.384503296,
  'cflop_std': 0.0,
  'comm_avg': 17.060879797604112,
  'comm_std': 0.011221766641588056},
 'non_iid_50_Vanilla-5-5': {'acc_avg': 0.8399886500201461,
  'acc_std': 0.0006838751471915724,
  'cflop_avg': 5.36758272,
  'cflop_std': 0.0,
  'comm_avg': 92.1249117112265,
  'comm_std': 0.07478870946063389},
 'non_iid_50_CESL-Random-6-10': {'acc_avg': 0.8355607119326814,
  'acc_std': 0.000905189070230315,
  'cflop_avg': 15.0484

In [5]:
compute_acc_per_comm_per_flop(metric_dict)

{'non_iid_50_CESL-3-5': 0.00962304299680502,
 'non_iid_50_CESL-No-Head-5-5': 0.0024304973207191203,
 'non_iid_50_CESL-Random-3-5': 0.00958609708588818,
 'non_iid_50_Vanilla-5-5': 0.0016987035216601588,
 'non_iid_50_CESL-Random-6-10': 0.001167806216193967,
 'non_iid_50_SplitFed-5-5': 0.002433996325940836}

In [10]:
# fedweit = {"cflop_avg": 5.36758272, ""}

import pandas as pd
df = pd.DataFrame(metric_dict)
df = df.T
df

,acc_avg,acc_std,cflop_avg,cflop_std,comm_avg,comm_std
non_iid_50_CESL-3-5,0.882147,0.001721,5.384503,0.000000e+00,17.024835,0.013270
non_iid_50_CESL-No-Head-5-5,0.842103,0.003228,3.760128,4.440892e-16,92.144051,0.086196
non_iid_50_CESL-Random-3-5,0.880621,0.001668,5.384503,0.000000e+00,17.060880,0.011222
non_iid_50_Vanilla-5-5,0.839989,0.000684,5.367583,0.000000e+00,92.124912,0.074789
non_iid_50_CESL-Random-6-10,0.835561,0.000905,15.048455,0.000000e+00,47.546146,0.064846
non_iid_50_SplitFed-5-5,0.843877,0.002463,3.760128,4.440892e-16,92.205406,0.068684
